<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Building a ReAct Agent from Scratch with LangGraph**
In this lab, you'll explore the ReAct (Reasoning and Acting) agent framework, which combines reasoning and action in language models to solve complex problems. You'll learn how to implement a ReAct agent from scratch using LangChain and LangGraph, starting with simple reasoning patterns and progressing to more complex implementations with tool usage. By the end of the lab, you'll understand how ReAct agents interleave thinking and acting to tackle multi-step problems that require external information or computation.


Estimated time needed: **90** minutes


## __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="#Importing-Required-Libraries">Importing Required Libraries</a></li>
        </ol>
    </li>
    <li>
        <a href="#What-is-a-ReAct-Agent?">What is a ReAct Agent?</a>
        <ol>
            <li><a href="#Core-Components-of-ReAct">Core Components of ReAct</a></li>
                <ol>
                     <li><a href="#Reasoning">Reasoning</a></li>
                    <li><a href="#Acting">Acting</a></li>
                </ol>
        </ol>
    </li>
    <li><a href="#Search-Tool-API-Key-Setup">Search Tool API Key Setup</a></li>
    <li><a href="#The-ReAct-Prompt-Pattern">The ReAct Prompt Pattern</a></li>
        <ol>
            <li><a href="#Structure-of-a-ReAct-Prompt">Structure of a ReAct Prompt</a></li>
        </ol>
    <li><a href="#Observing-and-Further-Reasoning">Observing and Further Reasoning</a></li>
        <ol>
            <li><a href="#Repeat-or-Conclude">Repeat or Conclude</a></li>
        </ol>
    <li><a href="#Let's-Create-a-more-Complex-ReAct-Agent-from-Scratch-with-Tools-using-LangGraph">Let's Create a more Complex ReAct Agent from Scratch with Tools using LangGraph</a></li>
        <ol>
            <li><a href="#Define-graph-state">Define graph state</a></li>
        </ol>
    <li><a href="#Define-Tools">Define Tools</a></li>
        <ol>
            <li><a href="#Define-all-the-Tools-for-the-ReAct-Agent">Define all the Tools for the ReAct Agent</a></li>
            <li><a href="#External-Search-Integration">External Search Integration</a></li>
            <li><a href="#API-Key-Setup">API Key Setup</a></li>
        </ol>
    <li><a href="#Define-the-tools">Define the tools</a></li>
    <li><a href="#Define-the-model">Define the model</a></li>
    <li><a href="#Load-ReAct-Prompt-Template">Load ReAct Prompt Template</a></li>
    <li><a href="#Define-nodes-and-edges">Define nodes and edges</a></li>
    <li><a href="#Define-the-graph">Define the graph</a></li>
    <li><a href="#Use-ReAct-agent">Use ReAct agent</a></li>
    <li><a href="#ReAct-Agent-Execution-Summary">ReAct Agent Execution Summary</a></li>
        <ol>
            <li><a href="#Flow-Overview">Flow Overview</a></li>
                <ol>
                    <li><a href="#Input">Input</a></li>
                    <li><a href="#Step-by-Step-Reasoning">Step-by-Step-Reasoning</a></li>  
                    <li><a href="#System-Behavior">System Behavior</a></li>   
                    <li><a href="#Exercise---add-a-tool">Exercise - add a tool</a></li>
                </ol>
        </ol>
    <li><a href="#References">References</a></li>
    <li><a href="#Authors">Authors</a></li>
  </ol>


## Objectives

After completing this lab, you will be able to:

- Understand the core components of ReAct (Reasoning and Acting) agents.
- Implement Chain of Thought reasoning patterns to improve LLM problem-solving.
- Create custom tools that expand an agent's capabilities.
- Build a complete ReAct agent using the LangGraph framework.
- Design effective prompts that guide agents through reasoning and action cycles.
- Implement the full reasoning-action-observation loop for multi-step problem solving.


----


## Setup

For this lab, we will be using the following libraries:

*   [`langgraph`](https://python.langchain.com/docs/langgraph) for building and running agent workflows
*   [`langchain`](https://python.langchain.com/docs/) for creating language model applications and chains  
*   [`langchain-openai`](https://python.langchain.com/docs/integrations/llms/openai) for integrating with OpenAI's models
*   [`langchainhub`](https://github.com/langchain-ai/langchainhub) for accessing premade prompts and components
*   [`IPython.display`](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html) for rendering markdown and displaying outputs
*   [`typing`](https://docs.python.org/3/library/typing.html) for type annotations and improved code readability
*   [`duckduckgo-search`](https://github.com/deedy5/duckduckgo_search) for web search capabilities
*   [`langchain-community`](https://python.langchain.com/docs/integrations/) for accessing community-created tools and integrations


### Installing Required Libraries

Run the following to install the required libraries:


In [ ]:
%%capture
!pip install langgraph==0.3.34 langchain-openai==0.3.14 langchainhub==0.1.21 langchain==0.3.24 pygraphviz==1.14 langchain-community==0.3.23

### Importing Required Libraries
We will import the other libraries as we proceed. 


In [ ]:
from IPython.display import display, Markdown

## What is a ReAct Agent?

ReAct is an agent framework that combines reasoning and acting within large language models (LLMs) to solve complex problems through a structured approach. Unlike standard language models that generate responses in a single pass, ReAct agents follow an iterative process that alternates between thinking and taking actions.

### Core Components of ReAct

1. **Reasoning** – The agent thinks through the problem, considering what it knows and what it needs to find out.  
2. **Acting** – Based on its reasoning, the agent selects and executes an appropriate action or tool.  
3. **Observing** – The agent receives feedback based on the action (that is, tool output).  
4. **Further Reasoning** – The agent incorporates observations into its reasoning process.  
5. **Repeat or Conclude** – The agent either takes another action or concludes with an answer.

What makes ReAct powerful is the continuous feedback loop between reasoning and action, allowing the agent to gather information incrementally and adapt its approach as new information becomes available.

We can use the following image to illustrate the ReAct framework:

![Screenshot 2025-04-17 at 12.51.55 PM.png](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/d2zMISxLwsKW7FkIftP-gQ/Screenshot%202025-04-17%20at%2012-51-55%E2%80%AFPM.png) 

When a query is received, it is passed to the LLM, which begins reasoning about how to address it. If the query requires information beyond the LLM’s internal knowledge—such as real-time data or external resources—the LLM identifies this gap and decides to interact with the external environment (represented by the "Environment" box in the diagram) to retrieve the missing information.

From there, the LLM decides to call a tool (such as a calculator, a web search function, or a database lookup). That tool interacts with the environment, retrieves the required information, and returns it to the LLM. This is shown by the arrows connecting tools and the environment.

The LLM then observes the result, reasons again, and may repeat this process—calling additional tools if needed—until it has enough information to conclude with a final answer.

In short, the LLM does not simply guess—it thinks, acts, observes, and iterates as needed, making it more reliable, dynamic, and capable.


## Reasoning

Reasoning is the foundation of ReAct agents. Language models can perform explicit reasoning, and one of the simplest yet most powerful ways to enhance their performance on complex tasks is just to ask them to think step by step.

**Chain of Thought (CoT)** prompting was one of the first effective techniques developed for this. The core idea is simple: rather than asking the model for an answer directly, you guide it to reason step by step.

CoT prompting isn't about telling the model what to think but showing it how to think. By providing examples that walk through the reasoning process, you're effectively teaching the model to mimic that kind of logical thinking in its own responses.

In ReAct agents, this reasoning capability is critical. It allows the agent to plan what information it needs and what tools it should use to obtain that information.


Let's create an instance of the GPT-4o-mini model with minimal randomness (temperature = 0.1).


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(model="gpt-4.1-nano",temperature=0.1)

# API Disclaimer
This lab uses LLMs provided by OpenAI. This environment has been configured to allow LLM use without API keys so you can prompt them for **free (with limitations)**. With that in mind, if you wish to run this notebook **locally outside** of Skills Network's JupyterLab environment, you will have to configure your own API keys. Please note that using your own API keys means that you will incur personal charges.
### Running Locally
If you are running this lab locally, you will need to configure your own API keys. This lab uses the `ChatOpenAI` module from `langchain`. The local configuration is shown below with instructions to use your own **api_key**. **Replace all instances** with the completed module below throughout the lab.

<p style='color: red'><b>DO NOT run the following cell if you aren't running locally, it will cause errors.</b>


In [ ]:
# IGNORE IF YOU ARE NOT RUNNING LOCALLY
from langchain_ibm import ChatOpenAI

openai_llm = ChatOpenAI(
    model="gpt-4.1-nano",
    api_key="your openai api key here",
    temperature=0.1
)

Let’s define a message that may require a bit of interpretation.
We’ll use the human prompt: "What is 1+1 for robots?"
At first glance, the question seems simple and direct, but it can invite deeper reasoning depending on how the model interprets it. In this case, the model responds with the answer “2” and provides a brief explanation of why.


In [ ]:
message=HumanMessage(content="what is 1+1 for robots")
response = llm.invoke([message])
print(response.content)

Now we define a message that explicitly walks through a reasoning process using CoT prompting. The prompt begins by noting that robots process numbers in binary, then breaks down the steps leading to the binary result of 1 + 1. 

The model is provided with step-by-step logic and guided by a structured reasoning path before producing the final answer. When this message is passed to the model, it’s expected to complete the thought by providing the binary result—"10"—demonstrating how CoT prompting encourages deeper, more logical outputs.


In [ ]:
message = """Robots are computers, and computers process numbers in binary. So, let's think step by step.

Question: what is 2+3 for robots?
']]
Step 1: In binary, the digits are 0 and 1.
Step 2: 1 + 1 in binary is similar to decimal 1 + 1.
Step 3: In decimal, 1 + 1 = 2.
Step 4: In binary, the number 2 is written as 10.

Answer:"""
response = llm.invoke([message])

print(response.content)

In situations where you lack sufficient knowledge to reason through a problem, you can instruct the language model to engage in its own reasoning process. For [zero shot CoT](https://arxiv.org/abs/2201.11903) , by prompting the model with something like **"Let's think step by step,"** you encourage it to deliberate before responding. This approach leverages techniques like CoT prompting, which enhances the model's ability to handle complex tasks by guiding it to generate intermediate reasoning steps.


In [ ]:
message = """Let's think step by step

Question: what is 2+3 for robots?



Answer:"""
response = llm.invoke([message])

print(response.content)

We didn’t initially get the exact answer we wanted, but simply by adding a small piece of text to the prompt, we improved the model’s performance significantly. This is the "Reasoning" step in ReAct. Now, let’s move on to the next step.


## Acting 


Acting is the second key component of ReAct. While reasoning enables structured thinking, it's limited by the knowledge contained within the language model. Acting extends this by allowing the agent to retrieve new information or perform computations using external tools.

For example, let's consider asking about the current weather in Tokyo. No matter how well the language model is trained, it cannot know the real-time weather in Tokyo at this moment, since it was trained on data from an earlier point in time.

This is where tools come in, they allow the agent to break out of its knowledge limitations and access real-time information or specialized capabilities.


In [ ]:
message=HumanMessage(content="What's the weather like in Tokyo today?")
response = llm.invoke([message])
print(response.content)

This tool connects our agent to Tavily's search API, enabling it to retrieve information from the internet. This is a crucial capability that allows our agent to access up-to-date information beyond its training data, making it much more versatile when answering knowledge-based questions. Pay attention to the **@tool** decorator before defining the function.  


## Search Tool API Key Setup

We'll use Tavily search as our external research tool. You can get an API key at https://app.tavily.com/sign-in    

**Disclaimer:** Signing up for Tavily provides you with free credits, more than enough for this project's needs. If you require additional credits for further use, please add them at your own discretion.  

You need to copy the key from Tavily's API website and paste the key in " " in the code snippet below: os.environ["TAVILY_API_KEY"] = "**enter your Tavily API key here**"


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import tool
import os

os.environ["TAVILY_API_KEY"] = "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"

# Initialize the Tavily search tool
search = TavilySearchResults()

@tool
def search_tool(query: str):
    """
    Search the web for information using Tavily API.

    :param query: The search query string
    :return: Search results related to the query
    """
    return search.invoke(query)

# Note: This tool connects our agent to Tavily’s real-time search API, enabling retrieval
# of up-to-date web content. Ideal for queries requiring current information.


Let's create a system message that instructs the agent how to format its responses.


In [ ]:
from langchain.agents import initialize_agent, AgentType

# Initialize the agent with the tool
agent_with_searchtool = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,  # Enables tool usage
    verbose=True,
    handle_parsing_errors=True)

In [ ]:
# from langgraph.prebuilt import create_react_agent

# agent = create_react_agent(
#     model=llm,
#     tools=[search_tool],
#     prompt="You are a helpful assistant that can search the web to answer questions.",
#     debug=True  # Enables verbose output for debugging
# )

Let's try the question again.


In [ ]:
# Run the agent with a query that should trigger the tool
response = agent_with_searchtool.run("What's the weather like in Tokyo today?")

# Output the response
response

You can see that we get a much better, and real-time, response.


In [ ]:
display(Markdown(response))

# The ReAct Prompt Pattern

Now that we've sketched the basics of a simple ReAct problem with a basic tool, let's look at the prompt pattern in detail. The ReAct prompt is a crucial component of the ReAct agent framework, designed to guide the language model in producing structured reasoning and action steps.

#### Structure of a ReAct Prompt

A ReAct prompt typically follows this structure:

1. **Available Tools**: List the tools the agent can useing-action cycle.






---

   
   ```python
   You have access to the following tools:
   1. Search: Search for information on the web.
   2. Calculator: Perform mathematical calculations.
   ```
---

2. **Step Instructions**: Define the expected reasoning process.

--- 
   ```python
   For each step:
   1. Think about what you know and what you need to find out.
   2. If you need more information, use a tool.
   3. Format tool usage as: Action: tool_name[parameters].
   4. After receiving an observation, continue reasoning.
   5. When you have enough information, provide your final answer.
   ```
---


3. **Example Demonstration**: Show a complete example of the reasoning-acting cycle.

---   
   ```python
   Question: What is the population of Paris multiplied by 2?
   Thought: I need to find the population of Paris first.
   Action: Search[population of Paris France].
   Observation: Paris has a population of approximately 2.16 million people.
   Thought: Now I need to multiply this number by 2.
   Action: Calculator[2.16 * 2]
   Observation: 4.32
   Answer: 4.32 million
   ```
---
   

4. **Current Query and Agent Scratchpad**: The actual question and a working area where the agent's thoughts, actions, and observations are recorded as it works through the problem. The scratchpad serves as the agent's memory, accumulating all steps of the reasoning process.

---
   
   ```python
   Question: {question}
   {agent_scratchpad}
   ```
---


Let's define a function (`my_message`) that takes in a question and creates a prompt for a ReAct agent. We will include some general outputs including how the AI should behave.  Most notably the LLM was told to stop its thoughts. 


In [ ]:
def my_message(question: str) -> str:
    return f"""
You are a thoughtful AI assistant tasked with solving the following question:

**{question}**

You will reason through the problem step by step. I will feed your previous thoughts back to you in future turns so that you can reflect and continue.

Use this format for each response:

---

**Thought:** [your reasoning and reflection here]

---

Do not give a final answer yet unless explicitly asked.  
If you see your own previous reasoning repeated as input, that means you should **continue expanding or improving** your thinking — not finalize.

Only when I explicitly ask you to provide a final answer, respond with:

**Final Answer:** [your answer]  
**DONE**
"""


Now, let's select a question to ask our LLM:


In [ ]:
my_prompt =my_message('Give me a list of warm-weather vacation locations and select the best one based on other factors')
print(my_prompt)

Finally, we need to incorporate  the scratchpad that serves as the agent's memory, accumulating all steps of the reasoning process.


## Observing and Further Reasoning

We know that incorporating observation and iterative reasoning can significantly enhance the effectiveness of an LLM. While reasoning alone enables structured thinking, acting gives the model access to real-time data or external knowledge, expanding the range of problems it can solve.

However, the true boost in performance comes from the **feedback** loop—observing the result of an action and applying further reasoning based on this new information. This loop allows the LLM to refine its understanding, correct course, and approach the problem with increasing precision. This is the **Repeat or Conclude** in the ReAct framework.


### Repeat or Conclude

The Repeat or Conclude step captures the essence of the ReAct loop — **the idea is that the LLM feeds its own output back into itself to continue reasoning**. In this example, we implement this using a 'for' loop, where the model is called multiple times, each time receiving its own previous output as part of the new input.

Each time through the loop, the model produces a new reasoning trace — an intermediate thought that helps build toward the final solution. These traces are stored in a variable called **scratch_pad**, which accumulates the model’s reasoning over time and acts as a running memory.

Although in this simplified example, the LLM is just reflecting on its own thoughts, this same loop structure could be extended to include tool use between iterations.

Eventually, when the LLM determines that it has nothing further to add, it concludes it by outputing DONE. Here we limit the **max iterations** to four.


In [ ]:
scratch_pad = ""
max_iterations = 4

for i in range(max_iterations):
    display(Markdown(f"### 🔁 Iteration {i + 1}"))

    # Initial message formatting
    if i == 0:
        user_input = my_message("Book me a vacation to a warm destination.")
    else:
        user_input = scratch_pad

    # Run the agent
    response = agent_with_searchtool.run(user_input)

    # Format and display the output cleanly
    display(Markdown(f"**📩 LLM Response:**\n\n```\n{response.strip()}\n```"))

    # Check for stopping condition
    if "DONE" in response:
        display(Markdown("✅ **Agent signaled DONE. Ending loop.**"))
        break

    # Append to scratch pad
    scratch_pad += f"\n\nIteration {i + 1}:\n{response.strip()}"


In [ ]:
display(Markdown(scratch_pad))

# Let's Create a more Complex ReAct Agent from Scratch with Tools using LangGraph


If you're building a ReAct-style agent, a simple **for loop** is a great way to get started. It's easy to control, ideal for testing ideas, and allows you to feed the model’s output back into itself step by step. However, as things get more complex—for example, when it becomes **unclear** whether the output is coming from a tool or is part of the agent’s reasoning process—that simplicity can turn into a limitation.

That’s where LangGraph comes in. It lets you structure your agent as a flow of reasoning steps, with built-in support for looping, memory, and a cleaner architecture.

**In short**: use a 'for loop' while prototyping or learning, and switch to LangGraph when you're ready for something more scalable and maintainable.






In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

## Define graph state 
We are going to define the custom ReAct state in this example, which will contain the user's query, a list of messages, and the `agent_outcome`.
  
For your specific use case, feel free to add any other state keys that you need.  

The `agent_outcome` field stores the result of the agent's reasoning process. This can be either an `AgentAction` (indicating the agent has decided to perform an action using a tool) or an `AgentFinish` (indicating the agent has arrived at a final answer).

Including `agent_outcome` in your state allows you to:

- Control Workflow: Determine the next step in your LangGraph workflow based on whether the agent has finished reasoning or needs to perform an action.

- Debugging: Inspect the agent's decisions at each step, which is invaluable for debugging and understanding agent behavior.

- State Management: Maintain a clear and structured state that reflects the agent's current status.

---


The `input` field represents the original user query or instruction. Keeping this in the state is useful for:  
  
- Contextual Awareness: Ensuring that the agent's actions and decisions are always grounded in the original user input.  

- Logging and Auditing: Facilitating logging of user interactions for auditing or analytics purposes.  
 
- Reusability: Allowing different parts of your workflow to access the original input without needing to pass it explicitly.

---

Last but not least, we need to keep track of the full conversation history between the user, the agent, and any tools that are used. This is done through the messages field.  
  
The messages field represents the running chat history, including:  

- HumanMessage (user input)  
  
- AIMessage (agent's thoughts and tool calls)  
    
- ToolMessage (tool responses)  
  
- SystemMessage (instructions to the LLM) - optionally  
  
In LangGraph, the messages field is annotated with a **reducer**, in this case, add_messages, to allow it to be updated incrementally and intelligently.  

--- 

**Why use add_messages as a reducer?**  
  
By default, LangGraph state updates **overwrite** previous values unless you use a **reducer** function. add_messages is a prebuilt smart reducer that allows:

- Appending new messages during each step of the agent's execution.  
  
- Overwriting existing messages with the **same message ID** (e.g., retrying or correcting a tool call).  
  
- Human-in-the-loop editing, where a user or external process can revise the conversation history without causing duplication.  
  
This is more powerful and safer than using operator.add, which simply appends new items blindly (potentially duplicating entries).  
  
Together, these three fields — input, agent_outcome, and messages — provide a robust foundation for any ReAct-style agent built with LangGraph. 
  


In [ ]:
import operator
from typing import Annotated, TypedDict, Union, Sequence, List, Tuple
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    """
    This defines the structure of our agent's state:
    - input: The user's query or instruction
    - agent_outcome: What the agent decides to do (take an action or provide final answer)
    - messages: The conversation history
    - intermediate_steps: A history of (action, observation) pairs used for scratchpad prompting
    """
    # add_messages is a reducer
    # See https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers
    
    input: str
    agent_outcome: Union[AgentAction, AgentFinish, None]
    messages: Annotated[Sequence[BaseMessage], add_messages]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]



# Define Tools


## Define all the Tools for the ReAct Agent

We had defined the `search tool` before, but to build a more complex ReAct agent we require  more tools for the agent to choose from during its action phase. Since the ReAct pattern emphasizes reasoning before acting, we need to define these tools upfront so the agent can reason about which tools to use and how to use them effectively.  

Here, we also define a `recommend_clothing` tool. The agent will use it when asked to recommend proper clothing for a weather. This is just an example. You can add as many tools as your agent required here. Make sure to include what the tool does inside the docstrings under tool definition so the agent can understand what the tool does and when to use it, for example :  

    """
    Suggest what to wear based on weather conditions.

    :param weather: A brief description of the weather (e.g. "Overcast, 64.9°F")
    :return: A recommendation on what to wear
    """



## External Search Integration

Let's set up the Tavily search tool that will allow our agent to perform external research.
  
This code sets up a web search tool using the Tavily API:

`TavilySearchAPIWrapper()` creates a basic search interface that:
* Makes HTTP requests to Tavily's API endpoints.
* Handles authentication using your API key.
* Processes raw search results into readable text.
* Returns plaintext summaries of search results.

The output is a dictionary where:
* `url`: Contains the source webpage URL.
* `content`: Contains the relevant text excerpt from the webpage.


## API Key Setup

We'll use Tavily search as our external research tool. You can get an API key at https://app.tavily.com/sign-in    

**Disclaimer:** Signing up for Tavily provides you with free credits, more than enough for this project's needs. If you require additional credits for further use, please add them at your own discretion.  



In [ ]:
os.environ["TAVILY_API_KEY"] = "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"

## Define the tools
This function uses a web search engine (like Tavily or DuckDuckGo) to retrieve and return relevant results for a given query string.


In [ ]:
# External tools
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import tool
import os

search = TavilySearchResults()

# Search Tool
@tool
def search_tool(query: str) -> str:
    """
    Search the web for information using a search engine like Tavily or DuckDuckGo.

    :param query: The search query string
    :return: Search results related to the query
    """
    return search.invoke(query) 


# Note: This tool connects our agent to Tavily’s real-time search API, enabling retrieval
# of up-to-date web content. Ideal for queries requiring current information.



This function analyzes a weather description string and returns a clothing recommendation by checking for specific keywords or temperatures—for example, if it finds words such as 'snow' or 'freezing', it suggests wearing warm clothing.


In [ ]:

@tool
def recommend_clothing(weather: str) -> str:
    """
    Returns a clothing recommendation based on the provided weather description.

    This function examines the input string for specific keywords or temperature indicators 
    (e.g., "snow", "freezing", "rain", "85°F") to suggest appropriate attire. It handles 
    common weather conditions like snow, rain, heat, and cold by providing simple and practical 
    clothing advice.

    :param weather: A brief description of the weather (e.g., "Overcast, 64.9°F")
    :return: A string with clothing recommendations suitable for the weather
    """
    weather = weather.lower()
    if "snow" in weather or "freezing" in weather:
        return "Wear a heavy coat, gloves, and boots."
    elif "rain" in weather or "wet" in weather:
        return "Bring a raincoat and waterproof shoes."
    elif "hot" in weather or "85" in weather:
        return "T-shirt, shorts, and sunscreen recommended."
    elif "cold" in weather or "50" in weather:
        return "Wear a warm jacket or sweater."
    else:
        return "A light jacket should be fine."

The tools can be added to a list to be used as input into the agent.


In [ ]:
tools = [search_tool, recommend_clothing]


The agent will also need the tool names, they are extracted using: 


In [ ]:
tools_by_name = {tool.name: tool for tool in tools}
print(tools_by_name)

## Define the model
Next, we will define the tools and model we will use for our graph.   
  


In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4", temperature=0)


## Load ReAct Prompt Template


#### Loading the ReAct Prompt from the Hub

Now that we have our tools and language model set up, the next step is to obtain the ReAct prompt template. This prompt guides the language model to follow the Reasoning–Acting–Observing pattern that defines the ReAct approach. You'll notice that it's similar to the prompt we discussed earlier.  

The prompt must have input keys:  
            - `tools`: contains descriptions and arguments for each tool.  
            -`tool_names`: contains all tool names.  
            -`agent_scratchpad`: contains previous agent actions and tool outputs as a string.  


In [ ]:
from langchain import hub

react_prompt = hub.pull("hwchase17/react")
print(react_prompt)  # this shows the full react_prompt

print("=" * 40)
print("📄 ReAct Prompt Template")
print("=" * 40)
print(react_prompt.template)
print("=" * 40)


The `hwchase17/react` prompt is a well-designed template specifically crafted for ReAct agents. It contains the necessary structure to:
- Guide the model to think through problems step by step.
- Format its actions to invoke the appropriate tools.
- Process observations from the tool calls.
- Make decisions based on accumulated information.


In [ ]:
from langchain import hub
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.runnables import RunnableConfig

# Load ReAct prompt from LangChain Hub
react_prompt = hub.pull("hwchase17/react")


def format_intermediate_steps(intermediate_steps):
    """Convert intermediate steps into a string for the prompt scratchpad."""
    scratchpad = ""
    for action, observation in intermediate_steps:
        scratchpad += f"Thought: {action.log}\n"
        scratchpad += f"Action: {action.tool}\n"
        scratchpad += f"Action Input: {action.tool_input}\n"
        scratchpad += f"Observation: {observation}\n"
    return scratchpad



def call_model(state: AgentState, config: RunnableConfig):
    # We have to format tool names as a string for the prompt (e.g., "search_tool, recommend_clothing")
    # Get tools from config
    tools = config["configurable"].get("tools", []) # configurable is an Arbitrary dictionary you define (e.g., tool list)
    # .get() won’t raise an error if the key doesn’t exist — it just returns a fallback instead, [] in this case. 
    

    # Prepare tool names for prompt
    tool_names = ", ".join(tool.name for tool in tools)
    
    
    # Format the scratchpad (past reasoning steps)
    scratchpad = format_intermediate_steps(state.get("intermediate_steps", []))

    # Format the ReAct-style prompt
    formatted_prompt = react_prompt.format(
        input=state["input"],
        tools=tool_names,
        tool_names=tool_names,
        agent_scratchpad=scratchpad
    )

    # Use it as a HumanMessage to feed into the model
    # giving the LLM a formatted string that includes the question and context — 
    # i.e., what the "user" might say and how the model should behave in response.
    prompt_message = HumanMessage(content=formatted_prompt) 

    # Call the model with prompt only (message history is embedded in the prompt text)
    response: AIMessage = model.invoke([prompt_message], config)
  

    # Determine agent outcome
    if response.tool_calls:
        agent_outcome = AgentAction(
            tool=response.tool_calls[0]["name"],
            tool_input=response.tool_calls[0]["args"],
            log=response.content or "",
        )
    else:
        agent_outcome = AgentFinish(
            return_values={"output": response.content},
            log="Agent has determined the final answer.",
        )
        print("we ard done",response.content)

    return {
        "messages": [response],
        "agent_outcome": agent_outcome,
        "agent_scratchpad": scratchpad
    }


In [ ]:
    # Define the user query and initial state
user_query = "What’s the weather in Zurich, and what should I wear check the temperture?"

fist_input = {
    "input": user_query,
    "messages": [HumanMessage(content=user_query)],
    "intermediate_steps": []  # this is mandatory for scratchpad to work
}


In [ ]:
scratchpad = format_intermediate_steps(fist_input.get("intermediate_steps", []))
scratchpad

In [ ]:
tool_names = ", ".join(tool.name for tool in tools)
tool_names 

In [ ]:
formatted_prompt = react_prompt.format(
        input=fist_input ["input"],
        tools=tool_names,
        tool_names=tool_names,
        agent_scratchpad=scratchpad
    )

In [ ]:
print(formatted_prompt)

LangGraph injects the tools into each node's function call, and if the node is calling a model (such as your call_model), it also wraps the model with those tools using .with_config(...) behind the scenes.


In [ ]:
model_tool = model.bind_tools(tools)

In [ ]:
response: AIMessage =model_tool.invoke([HumanMessage(formatted_prompt)])
response

In [ ]:
response.tool_calls


In [ ]:
 tools

## Define nodes and edges  
Next, let's define our nodes and edges.  
  
Each node should:

- Preserve and update messages.
- Set agent_outcome based on the model's decision (optional). 
- Keep input untouched.
    
You might consider adding a new node to produce structured outputs or trigger external actions like advanced calculations, or creating a calendar event. Or perhaps you want to modify how the `call_model` node behaves, or adjust the logic in `should_continue` to decide when tools should be used. With LangGraph, it's easy to tailor this foundational structure to fit your unique workflow.


In [ ]:
import json
from langchain_core.messages import ToolMessage, SystemMessage
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import AIMessage

tools_by_name = {tool.name: tool for tool in tools}


#Define our tool node 


def tool_node(state: AgentState):
    outputs = []
    new_steps = []

    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])

        # Append ToolMessage to message list
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )

        # Add intermediate step
        new_steps.append((
            AgentAction(
                tool=tool_call["name"],
                tool_input=tool_call["args"],
                log=state["messages"][-1].content or "",
            ),
            str(tool_result)
        ))

    return {
        "messages": outputs,
        "intermediate_steps": new_steps  # 👈 critical!
    }



def should_continue(state: AgentState) -> str:
    if isinstance(state["agent_outcome"], AgentFinish):
        return "end"
    return "continue"


## Define the graph  
Now that we have defined all of our nodes and edges, we can define and compile our graph. Depending on if you have added more nodes or different edges, you will need to edit this to fit your specific use case:


In [ ]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("reason", call_model)
workflow.add_node("act", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("reason")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `reason`.
    # This means these are the edges taken after the `agent` node is called.
    "reason",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `act`, then we call the tool node.
        "continue": "act",
        # Otherwise we finish.
        "end": END,
    },
)




# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("act", "reason")

# Now we can compile and visualize our graph
graph = workflow.compile()



In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

## Use ReAct agent  
Now that we have created our ReAct agent, let's actually put it to the test!


In [ ]:
# Sample execution of the message-based ReAct agent using LangGraph #working version
from langchain.schema import HumanMessage


if __name__ == "__main__":
    print("🧠 Initializing ReAct Agent with LangGraph")
    
    

    # Define the user query and initial state
    user_query = "What’s the weather in Zurich, and what should I wear?"

    inputs = {
    "input": user_query,
    "messages": [HumanMessage(content=user_query)],
    "intermediate_steps": []  # this is mandatory for scratchpad to work
}

    # Run the agent graph synchronously
    final_state = graph.invoke(
    inputs,
    config={"configurable": {"tools": tools}}  #  Pass tools here
    )
    #final_state = graph.invoke(inputs)

    # Extract and display the final answer (if available)
    print("\n✅ Final Answer:")
    finish = final_state.get("agent_outcome")
    if isinstance(finish, AgentFinish):
        print(finish.return_values["output"])
    else:
        print("No final answer was returned.")

    # Print the full reasoning trace from messages
    print("\n🔎 Reasoning Trace:")
    for msg in final_state["messages"]:
        try:
            msg.pretty_print()
        except AttributeError:
            print(msg)


## ReAct Agent Execution Summary

### Flow Overview
The agent successfully executed a multi-step reasoning task using LangGraph and ReAct prompting.
Below is a breakdown of the workflow:

### Input
**User query:**  
*"What’s the weather in Zurich, and what should I wear?"*

### Step-by-Step Reasoning

1. **First tool call:**  
   - **Tool:** `search_tool`  
   - **Input:** `"Current weather in Zurich"`  
   - **Observation:** `"The current weather in Zurich is 10 degrees Celsius with light rain."`

2. **Second tool call:**  
   - **Tool:** `recommend_clothing`  
   - **Input:** `"10 degrees Celsius, light rain"`  
   - **Observation:** `"For 10 degrees Celsius with light rain, it is recommended to wear a medium-weight jacket, 
      a sweater, long pants, and waterproof shoes. An umbrella or a raincoat would also be useful."`

3. **Final reasoning and answer:**
   - The agent concluded its thought process.
   - **Final Answer:**  
     *"The current weather in Zurich is 10 degrees Celsius with light rain. It is recommended to wear a medium-weight 
     jacket, a sweater, long pants, and waterproof shoes. An umbrella or a raincoat would also be useful."*

### System Behavior

| Component               | Description                                                                                   | Status |
|------------------------|-----------------------------------------------------------------------------------------------|--------|
| Input message           | Parsed correctly and injected into initial state                                              | ✅ Passed |
| First tool execution    | Weather data retrieved from search tool                                                      | ✅ Passed |
| Second tool execution   | Clothing advice generated based on weather                                                   | ✅ Passed |
| Scratchpad generation   | Thought, action, input, and observation logs formatted into a coherent history string         | ✅ Passed |
| Agent outcome tracking  | `AgentAction` and `AgentFinish` returned and handled appropriately                            | ✅ Passed |
| Reasoning trace output  | All intermediate steps and final result printed in readable format                           | ✅ Passed |

This confirms a correct and fully functional ReAct implementation using LangGraph
with tool chaining and reasoning history tracking. You can also define additional tools such as translation or summarization.


### Exercise - add a tool 

Add a temperature conversion tool to convert from Fahrenheit to Celcius and vice versa. Then test your ReAct agent with the following prompt:  
"What’s the weather in Tokyo today, what should I wear, and can you convert the temperature to Fahrenheit?"  


In [ ]:
df = # TODO

<details>
    <summary>Click here for the solution</summary>

```python

@tool
def convert_temperature(temperature: float, unit: str) -> str:
    """
    Converts temperature between Celsius and Fahrenheit.
    
    :param temperature: The numeric temperature to convert.
    :param unit: 'C' to convert to Celsius, 'F' to convert to Fahrenheit.
    :return: Converted temperature as a string.
    """
    unit = unit.upper()
    if unit == "C":
        converted = (temperature - 32) * 5 / 9
        return f"{temperature}°F is {converted:.1f}°C"
    elif unit == "F":
        converted = (temperature * 9 / 5) + 32
        return f"{temperature}°C is {converted:.1f}°F"
    else:
        return "Invalid unit. Please use 'C' or 'F'."

tools.append(convert_temperature)
tools_by_name[convert_temperature.name] = convert_temperature

##


```

</details>


## References 

- https://langchain-ai.github.io/langgraph/how-tos/react-agent-from-scratch/

- https://www.ibm.com/think/topics/react-agent


## Authors


[Faranak Heidari](https://www.linkedin.com/in/faranakhdr/) is a data scientist and GenAI developer in IBM. 


Kunal Makwana is a software developer in IBM. 


## Change Log


<details>
    <summary>Click here for the changelog</summary>

|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2025-06-24|0.4|Mercedes Schneider|QA pass with edits|
|2025-06-24|0.3|Steve Ryan|ID review and format/typo fixes|
|2024-02-23|0.2|Elio Di Nino|Update library documentation|
|2020-07-17|0.1|Sam|Create lab template|

</detials>


Copyright © IBM Corporation. All rights reserved.
